<a href="https://colab.research.google.com/github/VictorOPreuss/MeLi-Data-Challenge-2020/blob/main/MeLi_Data_Challenge_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bibliotecas**

In [4]:
!sudo apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/

In [5]:

# Install spark-related dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz

!pip install -q findspark
!pip install pyspark
# Set up required environment variables

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"


In [3]:
# Tools we need to connect to the Spark server, load our data,
# clean it and prepare it
from pyspark import SparkContext
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.sql.functions import *
from pyspark.sql.functions import monotonically_increasing_id 

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from matplotlib import rcParams
rcParams['figure.figsize'] = 16, 8

from scipy.stats import kurtosis, skew
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.mixture import GaussianMixture as GMM
from sklearn.model_selection import train_test_split

import os
import requests 
import zipfile
from google.colab import files

# settings
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 200)

# **Funções**

In [6]:
# Function to calculate missing values by column

def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

# Function to calculate % of values in a column

def percent_value_counts(df, feature):
    """This function takes in a dataframe and a column and finds the percentage of the value_counts"""
    percent = pd.DataFrame(round(df.loc[:,feature].value_counts(dropna=False, normalize=True)*100,2))
    ## creating a df with th
    total = pd.DataFrame(df.loc[:,feature].value_counts(dropna=False))
    ## concating percent and total dataframe

    total.columns = ["Total"]
    percent.columns = ['Percent']
    return pd.concat([total, percent], axis = 1)

# Function to keep columns with < X% missing values

def drop_features(df, percentage_missing_values): # keep columns with % of missing values < percentage_missing_values

  df_droped = df[df.columns[df.isnull().mean() <= (percentage_missing_values/100)]]
  
  return df_droped

# Function Histograms

def histograms(df, column):
  
  return df[column].plot(kind='hist', bins=100, figsize=(12,6), facecolor='grey',edgecolor='black')

# Function Kurtosis and Skewness

def kurt_and_skew(df, column):
  
  return print('Kurtosis:', kurtosis(df[column], nan_policy='omit'), 'Skew:', skew(df[column], nan_policy='omit'))

# Function for KNN model-based imputation of missing values using features without NaN as predictors

def impute_model_basic(df):
  cols_nan = df.columns[df.isna().any()].tolist()    
  cols_no_nan = df.columns.difference(cols_nan).values            
  for col in cols_nan:
      test_data = df[df[col].isna()]
      train_data = df.dropna()
      knr = KNeighborsRegressor(n_neighbors=5).fit(train_data[cols_no_nan], train_data[col])
      df.loc[df[col].isna(), col] = knr.predict(test_data[cols_no_nan])
  return df

# Functions GMM article

def draw_ellipse(position, covariance, ax=None, **kwargs):
    """Draw an ellipse with a given position and covariance"""
    ax = ax or plt.gca()
    # Convert covariance to principal axes
    if covariance.shape == (2, 2):
        U, s, Vt = np.linalg.svd(covariance)
        angle = np.degrees(np.arctan2(U[1, 0], U[0, 0]))
        width, height = 2 * np.sqrt(s)
    else:
        angle = 0
        width, height = 2 * np.sqrt(covariance)
    
    # Draw the Ellipse
    for nsig in range(1, 4):
        ax.add_patch(Ellipse(position, nsig * width, nsig * height,
                             angle, **kwargs))
        
def plot_gmm(gmm, X, label=True, ax=None):
    ax = ax or plt.gca()
    labels = gmm.fit(X).predict(X)
    if label:
        ax.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis', zorder=2)
    else:
        ax.scatter(X[:, 0], X[:, 1], s=40, zorder=2)
    
    w_factor = 0.2 / gmm.weights_.max()
    for pos, covar, w in zip(gmm.means_, gmm.covariances_, gmm.weights_):
        draw_ellipse(pos, covar, alpha=w * w_factor)
    plt.title("GMM with %d components"%len(gmm.means_), fontsize=(20))
    plt.xlabel("U.A.")
    plt.ylabel("U.A.")
def SelBest(arr:list, X:int)->list:
    '''
    returns the set of X configurations with shorter distance
    '''
    dx=np.argsort(arr)[:X]
    return arr[dx]

def normalize_col(df_col):
    return (df_col-df_col.min())/(df_col.max()-df_col.min())

def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values

    for name in summary['Name'].value_counts().index:
        summary.loc[summary['Name'] == name, 'Entropy'] = round(stats.entropy(df[name].value_counts(normalize=True), base=2),2) 

    return summary

## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def CalcOutliers(df_num): 

    # calculating mean and std of the array
    data_mean, data_std = np.mean(df_num), np.std(df_num)

    # seting the cut line to both higher and lower values
    # You can change this value
    cut = data_std * 3

    #Calculating the higher and lower cut values
    lower, upper = data_mean - cut, data_mean + cut

    # creating an array of lower, higher and total outlier values 
    outliers_lower = [x for x in df_num if x < lower]
    outliers_higher = [x for x in df_num if x > upper]
    outliers_total = [x for x in df_num if x < lower or x > upper]

    # array without outlier values
    outliers_removed = [x for x in df_num if x > lower and x < upper]
    
    print('Identified lowest outliers: %d' % len(outliers_lower)) # printing total number of values in lower cut of outliers
    print('Identified upper outliers: %d' % len(outliers_higher)) # printing total number of values in higher cut of outliers
    print('Total outlier observations: %d' % len(outliers_total)) # printing total number of values outliers of both sides
    print('Non-outlier observations: %d' % len(outliers_removed)) # printing total number of non outlier values
    print("Total percentual of Outliers: ", round((len(outliers_total) / len(outliers_removed) )*100, 4)) # Percentual of outliers in points
    
    return

In [7]:
import findspark
findspark.init("spark-3.0.1-bin-hadoop2.7")# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

# **ETL**

In [8]:

! wget https://meli-data-challenge.s3.amazonaws.com/2020/train_dataset.jl.gz
! wget https://meli-data-challenge.s3.amazonaws.com/2020/test_dataset.jl.gz
! wget https://meli-data-challenge.s3.amazonaws.com/2020/item_data.jl.gz
! wget https://meli-data-challenge.s3.amazonaws.com/2020/sample_submission.csv


--2020-11-14 13:58:15--  https://meli-data-challenge.s3.amazonaws.com/2020/train_dataset.jl.gz
Resolving meli-data-challenge.s3.amazonaws.com (meli-data-challenge.s3.amazonaws.com)... 52.217.36.252
Connecting to meli-data-challenge.s3.amazonaws.com (meli-data-challenge.s3.amazonaws.com)|52.217.36.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 128022297 (122M) [application/x-gzip]
Saving to: ‘train_dataset.jl.gz’

train_dataset.jl.gz 100%[===================>] 122.09M  39.0MB/s    in 3.1s    

2020-11-14 13:58:19 (39.0 MB/s) - ‘train_dataset.jl.gz’ saved [128022297/128022297]

--2020-11-14 13:58:19--  https://meli-data-challenge.s3.amazonaws.com/2020/test_dataset.jl.gz
Resolving meli-data-challenge.s3.amazonaws.com (meli-data-challenge.s3.amazonaws.com)... 52.216.228.168
Connecting to meli-data-challenge.s3.amazonaws.com (meli-data-challenge.s3.amazonaws.com)|52.216.228.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54305870 

In [9]:

! gunzip train_dataset.jl.gz
! gunzip test_dataset.jl.gz
! gunzip item_data.jl.gz


In [10]:
train_path = '/content/train_dataset.jl'
test_path = '/content/test_dataset.jl'
item_data_path = '/content/item_data.jl'

In [11]:
train = spark.read.json(train_path, multiLine= False)
test = spark.read.json(test_path, multiLine= False)
item_data = spark.read.json(item_data_path)
sample_submission = pd.read_csv('sample_submission.csv')

In [12]:
train.printSchema()

root
 |-- item_bought: long (nullable = true)
 |-- user_history: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- event_info: string (nullable = true)
 |    |    |-- event_timestamp: string (nullable = true)
 |    |    |-- event_type: string (nullable = true)



In [13]:
test.printSchema()

root
 |-- user_history: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- event_info: string (nullable = true)
 |    |    |-- event_timestamp: string (nullable = true)
 |    |    |-- event_type: string (nullable = true)



In [14]:
item_data.printSchema()

root
 |-- category_id: string (nullable = true)
 |-- condition: string (nullable = true)
 |-- domain_id: string (nullable = true)
 |-- item_id: long (nullable = true)
 |-- price: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- title: string (nullable = true)



In [15]:
train_index = train.select("*").withColumn("id", monotonically_increasing_id())
test_index = test.select("*").withColumn("id", monotonically_increasing_id())

In [16]:
train_final = (train_index
        .select("item_bought", "id", 
explode("user_history").alias("new_user_history"))
        .select("item_bought", "id", "new_user_history.*"))

In [17]:
test_final = (test_index
        .select("id", explode("user_history").alias("new_user_history"))
        .select("id", "new_user_history.*"))

# **EDA & Prep - Baseline**

In [18]:
train_final.show(10)

+-----------+---+------------------+--------------------+----------+
|item_bought| id|        event_info|     event_timestamp|event_type|
+-----------+---+------------------+--------------------+----------+
|    1748830|  0|           1786148|2019-10-19T11:25:...|      view|
|    1748830|  0|           1786148|2019-10-19T11:25:...|      view|
|    1748830|  0|RELOGIO SMARTWATCH|2019-10-19T11:26:...|    search|
|    1748830|  0|           1615991|2019-10-19T11:27:...|      view|
|    1748830|  0|           1615991|2019-10-19T11:28:...|      view|
|    1748830|  0|           1615991|2019-10-19T11:28:...|      view|
|    1748830|  0|           1615991|2019-10-19T11:30:...|      view|
|    1748830|  0|           1615991|2019-10-19T21:51:...|      view|
|    1748830|  0|           1615991|2019-10-19T21:52:...|      view|
|    1748830|  0|           1615991|2019-10-19T21:52:...|      view|
+-----------+---+------------------+--------------------+----------+
only showing top 10 rows



In [42]:
test_final.show(10)

+---+----------+--------------------+----------+
| id|event_info|     event_timestamp|event_type|
+---+----------+--------------------+----------+
|  0|   1572239|2019-09-26T18:31:...|      view|
|  0|   1572239|2019-09-26T18:35:...|      view|
|  0|   1572239|2019-09-26T18:37:...|      view|
|  0|   1572239|2019-09-26T18:38:...|      view|
|  0|   1572239|2019-09-26T18:40:...|      view|
|  0|   1572239|2019-09-26T18:40:...|      view|
|  0|   1572239|2019-09-26T18:41:...|      view|
|  0|   1572239|2019-09-26T18:41:...|      view|
|  0|   1572239|2019-09-26T19:03:...|      view|
|  0|   1194894|2019-09-27T21:33:...|      view|
+---+----------+--------------------+----------+
only showing top 10 rows



In [19]:
item_data.show(10)

+-----------+---------+--------------------+-------+----------+----------+--------------------+
|category_id|condition|           domain_id|item_id|     price|product_id|               title|
+-----------+---------+--------------------+-------+----------+----------+--------------------+
|  MLM170527|      new|MLM-INDIVIDUAL_HO...| 111260|1150000.00|      null|Casa Sola En Vent...|
|  MLM151595|      new|     MLM-VIDEO_GAMES| 871377|   1392.83|  15270800|Resident Evil Ori...|
|    MLM7697|      new|          MLM-SKIRTS| 490232|    350.00|      null|Falda De Imitació...|
|    MLM9761|     used|  MLM-GRAPHICS_CARDS|1150706|   3200.00|      null|Powercolor Red De...|
|    MLM1652|     used|       MLM-NOTEBOOKS| 934912|   1599.00|      null|Laptop Hp Nx6320 ...|
|   MLM92472|      new|MLM-VEHICLE_ACCES...| 534737|    470.00|      null|Transmisor Fm Sin...|
|  MLM167442|      new|MLM-CELLPHONE_COVERS| 369182|    589.00|      null|Funda Cartera Cas...|
|    MLM1588|      new|MLM-WALL_AND_CEIL

In [20]:
train_final.dtypes

[('item_bought', 'bigint'),
 ('id', 'bigint'),
 ('event_info', 'string'),
 ('event_timestamp', 'string'),
 ('event_type', 'string')]

In [21]:
item_data.dtypes

[('category_id', 'string'),
 ('condition', 'string'),
 ('domain_id', 'string'),
 ('item_id', 'bigint'),
 ('price', 'string'),
 ('product_id', 'string'),
 ('title', 'string')]

In [22]:
item_data.createGlobalTempView("metadata")
train_final.createGlobalTempView("train")
test_final.createGlobalTempView("test")

In [92]:
train_query = spark.sql("""

SELECT 
  t.id,
  COUNT(DISTINCT t.event_info) AS unq_event_info,
  SUM(CASE WHEN t.event_type = 'view' then 1 else 0 end) AS sum_view,
  SUM(CASE WHEN t.event_type = 'search' then 1 else 0 end) AS sum_search,
  MIN(to_timestamp(t.event_timestamp, "yyyy-MM-dd'T'HH:mm:ss.SSS'-0400'")) AS first_timestamp,
  MAX(to_timestamp(t.event_timestamp, "yyyy-MM-dd'T'HH:mm:ss.SSS'-0400'")) AS last_timestamp,
  (UNIX_TIMESTAMP(MAX(to_timestamp(t.event_timestamp, "yyyy-MM-dd'T'HH:mm:ss.SSS'-0400'"))) - UNIX_TIMESTAMP(MIN(to_timestamp(t.event_timestamp, "yyyy-MM-dd'T'HH:mm:ss.SSS'-0400'")))) AS diff_max_min_time_stamp_sec,
  m.category_id,
  t.event_info,
  m.title,
  ROUND(AVG(m.price),2) AS avg_price,
  t.item_bought  
FROM global_temp.train t
LEFT JOIN global_temp.metadata m ON m.item_id = t.event_info
GROUP BY t.id, t.event_info, m.title, m.category_id, t.item_bought
SORT BY sum_view DESC, sum_search DESC

""")

In [93]:
train_query.show(100)

+-----------+--------------+--------+----------+--------------------+--------------------+---------------------------+-----------+----------+--------------------+---------+-----------+
|         id|unq_event_info|sum_view|sum_search|     first_timestamp|      last_timestamp|diff_max_min_time_stamp_sec|category_id|event_info|               title|avg_price|item_bought|
+-----------+--------------+--------+----------+--------------------+--------------------+---------------------------+-----------+----------+--------------------+---------+-----------+
|68719486401|             1|      55|         0|2019-10-09 17:40:...|2019-10-14 09:19:...|                     401989|   MLB23332|   1526724|Kit 4 Pares Tênis...|    149.9|    1040602|
|17179897946|             1|      31|         0|2019-09-30 12:54:...|2019-10-02 19:10:...|                     195390|  MLB135384|    817689|Pulseira Mi Band ...|    179.0|     817689|
| 8589941026|             1|      29|         0|2019-09-28 19:22:...|2019-0

In [67]:
test_query = spark.sql("""

SELECT 
  id,
  COUNT(DISTINCT event_info) AS unq_event_info,
  SUM(CASE WHEN event_type = 'view' then 1 else 0 end) AS sum_view,
  SUM(CASE WHEN event_type = 'search' then 1 else 0 end) AS sum_search,
  MIN(to_timestamp(event_timestamp, "yyyy-MM-dd'T'HH:mm:ss.SSS'-0400'")) AS first_timestamp,
  MAX(to_timestamp(event_timestamp, "yyyy-MM-dd'T'HH:mm:ss.SSS'-0400'")) AS last_timestamp,
  (UNIX_TIMESTAMP(MAX(to_timestamp(event_timestamp, "yyyy-MM-dd'T'HH:mm:ss.SSS'-0400'"))) - UNIX_TIMESTAMP(MIN(to_timestamp(event_timestamp, "yyyy-MM-dd'T'HH:mm:ss.SSS'-0400'")))) AS diff_max_min_time_stamp_sec
FROM global_temp.test
GROUP BY id
SORT BY sum_view DESC

""")

In [68]:
test_query.show(10)

+-----------+--------------+--------+----------+--------------------+--------------------+---------------------------+
|         id|unq_event_info|sum_view|sum_search|     first_timestamp|      last_timestamp|diff_max_min_time_stamp_sec|
+-----------+--------------+--------+----------+--------------------+--------------------+---------------------------+
|      15237|           140|     217|       241|2019-10-10 06:52:...|2019-10-16 19:24:...|                     563528|
|25769806291|           122|     183|        84|2019-09-30 14:01:...|2019-10-06 21:43:...|                     546130|
| 8589947960|            71|     124|        67|2019-10-24 20:14:...|2019-10-30 11:53:...|                     488349|
| 8589966965|            64|     122|        50|2019-10-02 18:24:...|2019-10-05 13:17:...|                     240813|
| 8589955165|            62|     115|       113|2019-09-28 06:15:...|2019-10-04 09:55:...|                     531645|
|34359740494|            39|     107|        46|